In [1]:
import numpy as np
from pyscf import gto, scf, df

In [120]:
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='6-311G', charge = 1)
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='pc-3', charge = 1)
#mol = gto.M(atom='C 0 0 0; O 0 0 1.128', basis='STO-3G', charge = 0)
#mol = gto.M(atom='H 0 0 0; H 0 0 0.740848149', basis='STO-3G', charge = 0)
mol = gto.M(atom="O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116", basis = "sto-3g")

In [121]:
#calculate nuclear repulsion energy
tot_nuc_energy=0
for i in range(0,len(mol.atom_coords())-1):
    for j in range(0,len(mol.atom_coords())):
        if j > i:
            print("i",i,"-- j",j)
            x=mol.atom_coord(i)-mol.atom_coord(j)
           # print("x",x)
            norm=np.linalg.norm(x)
           # print("norm",norm)
            nuc_energy=mol.atom_charge(i)*mol.atom_charge(j)/norm
            print("nuc_energy",nuc_energy)     
            tot_nuc_energy+=nuc_energy
print("tot_nuc_energy",tot_nuc_energy)

i 0 -- j 1
nuc_energy 4.419491196738508
i 0 -- j 2
nuc_energy 4.419491196738508
i 1 -- j 2
nuc_energy 0.3502397318948971
tot_nuc_energy 9.189222125371913


In [122]:
#get integrals and calculate core Hamiltonian

overlap_integrals = mol.intor('int1e_ovlp_sph')
#print("overlap_integrals")
#print(overlap_integrals)
kinetic_energy_integrals = mol.intor('int1e_kin_sph')
#print("kinetic_energy_integrals")
#print(kinetic_energy_integrals)
nuclear_attraction_integrals = mol.intor('int1e_nuc_sph')
#print("nuclear_attraction_integrals")
#print(nuclear_attraction_integrals)
two_electron_integrals = mol.intor('int2e_sph')
#print("two_electron_integrals")
#print(two_electron_integrals)
H_core = kinetic_energy_integrals + nuclear_attraction_integrals

In [123]:
#diagonalize overlap matrix
overlap_diag_procedure = np.linalg.eigh(overlap_integrals)
#get U and S^-0.5
U = overlap_diag_procedure[1]
s_inverse_sqrt = np.diag(1/overlap_diag_procedure[0])**0.5
#get transformation matrix X
X = np.matmul(U,np.matmul(s_inverse_sqrt,np.matrix.transpose(U)))

In [124]:
#set guess for P matrix, default core hamiltonian
P = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

In [125]:
#start iterations
counter = 0
maxiter = 100
energy_difference=10000
energy_last_iter=10000
energy=0

for counter in range(0,maxiter):
    G = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    G[i,j]+=P[k,l]*(two_electron_integrals[i,j,k,l]-0.5*two_electron_integrals[i,l,k,j])

    F = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    F = H_core + G
#    print(F)
    
    if counter >= 1:
        energy = 0
        for i in range(0,overlap_integrals.shape[0]):
            for j in range(0,overlap_integrals.shape[0]):
                energy += 0.5*(P[i,j]*(H_core[i,j]+F[i,j]))
        energy_difference = abs(energy-energy_last_iter)
        print('iteration {:<9}'.format(counter),"energy = {:<25}".format(energy), "difference = {:<23}".format(energy_difference))
        energy_last_iter = energy
    if energy_difference < 10**-8:
        print("energy converged")
        SCF_final_energy = energy + tot_nuc_energy
        print("final SCF energy including nuclear repulsion:",SCF_final_energy)
        break
    counter+=1    

    F_prime = np.matmul(np.matrix.transpose(X),np.matmul(F,X))
    F_diag_procedure = np.linalg.eigh(F_prime)
    
    C_prime = np.matmul(X,F_diag_procedure[1])
    
    P_new = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    
    for i in range(0,overlap_integrals.shape[0]):
        for j in range(0,overlap_integrals.shape[1]):    
            for a in range(0,mol.nelectron//2):
                P_new[i,j]+=2*C_prime[i,a]*C_prime[j,a]

    P = P_new

iteration 1         energy = -82.42145564194183        difference = 10082.421455641941     
iteration 2         energy = -84.13474060599256        difference = 1.7132849640507288     
iteration 3         energy = -84.15150299234627        difference = 0.016762386353704528   
iteration 4         energy = -84.15225184674064        difference = 0.0007488543943736659  
iteration 5         energy = -84.15234830500484        difference = 9.64582641955758e-05   
iteration 6         energy = -84.15236481052716        difference = 1.650552232490554e-05  
iteration 7         energy = -84.15236782267155        difference = 3.0121443899133737e-06 
iteration 8         energy = -84.1523683835041         difference = 5.608325466255337e-07  
iteration 9         energy = -84.15236848857609        difference = 1.0507199021958513e-07 
iteration 10        energy = -84.15236850829915        difference = 1.9723060518117563e-08 
iteration 11        energy = -84.15236851200356        difference = 3.7044145528

In [ ]:
#HeH+

STO-3G
iteration 7         energy = -3.9112755000406874       difference = 3.5873601866853733e-09 
STO-6G
iteration 7         energy = -3.944682131426069        difference = 4.717611190585558e-09  

6-31G
iteration 7         energy = -3.9531413124082446       difference = 2.4788495700533986e-09
G-31G**
iteration 7         energy = -3.966274032010168        difference = 3.9444496557905495e-09 
6-311G
iteration 7         energy = -3.960132124797542        difference = 1.9460095757040108e-09 
6-311G**
iteration 7         energy = -3.9726835051315375       difference = 3.6593732488654496e-09 



def2-SVP
iteration 7         energy = -3.967055803161621        difference = 4.2459431526253866e-09 
def2-TZVP
iteration 7         energy = -3.9726576432308645       difference = 3.6573504225145825e-09 
def2-QZVP
iteration 7         energy = -3.975303918837294        difference = 3.859934150085564e-09  


pcseg-1
iteration 7         energy = -3.9625337677038153       difference = 3.4461096198867835e-09 
pcseg-2
iteration 7         energy = -3.9745154194844305       difference = 4.017403743006298e-09  
pcseg-3
iteration 7         energy = -3.975333716944547        difference = 3.8931657897478544e-09 


pc-1
iteration 7         energy = -3.9625323446093272       difference = 3.4460296838290105e-09 
pc-2
iteration 7         energy = -3.974515581851391        difference = 4.017171928438756e-09  
pc-3
iteration 7         energy = -3.9753471915476237       difference = 3.847582696892005e-09  



cc-pVDZ
iteration 7         energy = -3.9670705482078885       difference = 4.235916506445392e-09  
cc-pVTZ
iteration 7         energy = -3.9746967846281955       difference = 3.992973951483236e-09  





In [ ]:
#CO
STO-3G
iteration 40        energy = -133.7427378748965        difference = 6.021394938215963e-09  
STO-6G
iteration 40        energy = -134.82148004689435       difference = 7.270188007169054e-09  



In [ ]:
#H2O

STO-3G
iteration 11        energy = -84.15236851200356        difference = 3.7044145528852823e-09 


In [74]:
#some coding for nuclear repulstion stuff

x=mol.atom_coord(1)-mol.atom_coord(0)
print(x)
norm=np.linalg.norm(x)
print(norm)
nuc_energy=mol.atom_charge(0)*mol.atom_charge(1)/norm
print(nuc_energy)

[0.        0.        1.4000001]
1.4000001015009693
0.7142856624995092


In [105]:
#some coding for nuclear repulstion stuff

tot_nuc_energy=0
for i in range(0,len(mol.atom_coords())-1):
    for j in range(0,len(mol.atom_coords())):
        if j > i:
            print("i",i,"-- j",j)
            x=mol.atom_coord(i)-mol.atom_coord(j)
           # print("x",x)
            norm=np.linalg.norm(x)
           # print("norm",norm)
            nuc_energy=mol.atom_charge(i)*mol.atom_charge(j)/norm
            print("nuc_energy",nuc_energy)     
            tot_nuc_energy+=nuc_energy
            print("tot_nuc_energy",tot_nuc_energy)


i 0 -- j 1
nuc_energy 22.51817918808511
tot_nuc_energy 22.51817918808511
